In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
import tensorflow as tf
from pathlib import Path
from time import strftime

In [2]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs
print('Tensorboard launched at localhost:6006')

Launching TensorBoard...

Tensorboard launched at localhost:6006


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [27]:
def get_run_dir(root='my_logs'):
    return Path(root) / strftime('run_%Y_%m_%d_%H_%M_%S')

In [28]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

In [29]:
dataset = pd.read_csv('dataset_all_features.csv')

In [30]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]

# Deleting categorical features

In [31]:
data.drop(columns=['FACEATTRIBUTES-BLUR-BLURLEVEL', 'FACEATTRIBUTES-EXPOSURE-EXPOSURELEVEL', 'FACEATTRIBUTES-GENDER',
                   'FACEATTRIBUTES-GLASSES', 'FACEATTRIBUTES-HAIR-INVISIBLE', 'FACEATTRIBUTES-MAKEUP-EYEMAKEUP',
                   'FACEATTRIBUTES-MAKEUP-LIPMAKEUP', 'FACEATTRIBUTES-NOISE-NOISELEVEL',
                   'FACEATTRIBUTES-ACCESSORIES', 'FACEID'],
          inplace=True)

# Splitting data

In [32]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling values to range 0 - 1

In [33]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)
data_test_mms = mms.fit_transform(data_test)
data_test_mms = pd.DataFrame(data_test_mms, columns=data_test.columns)

# Scaling input values to Gaussian distributions

In [34]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)

data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)
data_test_qt = quantile_transformer.fit_transform(data_test_mms)
data_test_qt = pd.DataFrame(data_test_qt, columns=data_test_mms.columns)

# Training

In [110]:
tf.random.set_seed(123)

n_inputs = data_train_qt.shape[1]

def relu_max(x):
    return tf.keras.activations.relu(x, max_value=9)

model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=n_inputs),
        tf.keras.layers.Dense(n_inputs, activation='relu'),
        tf.keras.layers.Dense(n_inputs, activation='relu'),
        tf.keras.layers.Dense(2, activation=relu_max)
    ]
)

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 89)                8010      
                                                                 
 dense_49 (Dense)            (None, 89)                8010      
                                                                 
 dense_50 (Dense)            (None, 2)                 180       
                                                                 
Total params: 16,200
Trainable params: 16,200
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [112]:
n_epochs = 50

run_dir = get_run_dir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

history = model.fit(data_train_qt, labels_train, epochs=n_epochs, batch_size= 50,
                    validation_data=(data_validation_qt, labels_validation), callbacks=[tensorboard_cb])

Epoch 1/50
365/365 [==============================] - 2s 4ms/step - loss: 5.0175 - mean_absolute_error: 1.8420 - val_loss: 5.8573 - val_mean_absolute_error: 2.0096
Epoch 2/50
365/365 [==============================] - 1s 4ms/step - loss: 4.0693 - mean_absolute_error: 1.6848 - val_loss: 5.7005 - val_mean_absolute_error: 1.9845
Epoch 3/50
365/365 [==============================] - 1s 4ms/step - loss: 4.0057 - mean_absolute_error: 1.6703 - val_loss: 5.7103 - val_mean_absolute_error: 1.9791
Epoch 4/50
365/365 [==============================] - 1s 3ms/step - loss: 3.9679 - mean_absolute_error: 1.6594 - val_loss: 5.8263 - val_mean_absolute_error: 2.0077
Epoch 5/50
365/365 [==============================] - 1s 3ms/step - loss: 3.9168 - mean_absolute_error: 1.6471 - val_loss: 5.6935 - val_mean_absolute_error: 1.9705
Epoch 6/50
365/365 [==============================] - 3s 7ms/step - loss: 3.8869 - mean_absolute_error: 1.6397 - val_loss: 5.7071 - val_mean_absolute_error: 1.9746
Epoch 7/50
365/3

In [113]:
predictions = model.predict(data_test_qt)

mae, mse, r2 = get_score(labels_test, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

268/268 [==============================] - 0s 1ms/step

Values for validation set:
MAE: 1.9643370734574488
MSE: 6.0689975445280435
RMSE: 2.4635335484884395
R2:  -0.03127502821219097


# Overfitting?

In [19]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=n_inputs)
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(2)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 89)]              0         
                                                                 
 normalization_2 (Normalizat  (None, 89)               179       
 ion)                                                            
                                                                 
 dense_5 (Dense)             (None, 2)                 180       
                                                                 
Total params: 359
Trainable params: 180
Non-trainable params: 179
_________________________________________________________________


In [20]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [21]:
norm_layer.adapt(data_train_qt)

In [22]:
n_epochs = 50

# run_dir = get_run_dir()
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation_qt, labels_validation))

Epoch 1/50
570/570 [==============================] - 3s 5ms/step - loss: 29.8727 - mae: 5.0167 - val_loss: 22200.0625 - val_mae: 147.6092
Epoch 2/50
570/570 [==============================] - 3s 5ms/step - loss: 24.4953 - mae: 4.4760 - val_loss: 5629.7378 - val_mae: 67.2040
Epoch 3/50
570/570 [==============================] - 2s 4ms/step - loss: 20.0869 - mae: 3.9724 - val_loss: 1477.4966 - val_mae: 36.1353
Epoch 4/50
570/570 [==============================] - 3s 4ms/step - loss: 16.4026 - mae: 3.5202 - val_loss: 2506.2991 - val_mae: 46.1507
Epoch 5/50
570/570 [==============================] - 2s 4ms/step - loss: 13.3421 - mae: 3.1322 - val_loss: 11326.8760 - val_mae: 105.5095
Epoch 6/50
570/570 [==============================] - 2s 4ms/step - loss: 10.8422 - mae: 2.8044 - val_loss: 5465.4497 - val_mae: 70.4654
Epoch 7/50
570/570 [==============================] - 2s 4ms/step - loss: 8.8383 - mae: 2.5298 - val_loss: 7358.6997 - val_mae: 72.2214
Epoch 8/50
570/570 [==================

In [23]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 0s 2ms/step

Values for validation set:
MAE: 1.9900330094556329
MSE: 5.703879559937679
RMSE: 2.388279623481656
R2:  0.0006626172185835166
